# Save predicted

In [1]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from torchvision import transforms

# Import your model here
from src.model import MobileNetUNet  # Adjust the import according to your project structure

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [2]:
def load_model(model_path, device):
    """Load model from checkpoint"""
    model = MobileNetUNet(img_ch=1, seg_ch=4, num_classes=4).to(device)
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'] if 'model_state_dict' in checkpoint else checkpoint)
    model.eval()
    return model

In [4]:
def predict_masks(model, test_dir, output_dir):
    """Predict segmentation masks for all images in the test directory and save them."""
    os.makedirs(output_dir, exist_ok=True)

    # Class names for saving output
    class_names = ['No_Tumor', 'Glioma', 'Meningioma', 'Pituitary']
    for class_name in class_names:
        os.makedirs(os.path.join(output_dir, class_name), exist_ok=True)

    # Define normalization transform
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])

    true_classes = []
    pred_classes = []

    for class_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(test_dir, 'image', class_name)
        image_files = [f for f in os.listdir(class_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

        for img_file in tqdm(image_files, desc=f"Processing {class_name}"):
            img_path = os.path.join(class_dir, img_file)
            img = Image.open(img_path).convert('L').resize((256, 256))
            img_tensor = transform(img).unsqueeze(0).to(device)

            with torch.no_grad():
                seg_output, cls_output = model(img_tensor)
                seg_pred = torch.argmax(seg_output, dim=1).squeeze().cpu().numpy()
                cls_pred = torch.argmax(cls_output, dim=1).item()

            # Save segmentation mask to the folder corresponding to the true class
            output_path = os.path.join(output_dir, class_name, f"{os.path.splitext(img_file)[0]}.png")
            scaled_mask = (seg_pred * 85).astype(np.uint8)
            mask_img = Image.fromarray(scaled_mask)
            mask_img.save(output_path)

            true_classes.append(class_idx)
            pred_classes.append(cls_pred)
    
    return true_classes, pred_classes

def calculate_recall(true_classes, pred_classes):
    """Calculate recall based on true and predicted classes."""
    recall = {}
    for i in range(len(set(true_classes))):
        tp = sum((np.array(true_classes) == i) & (np.array(pred_classes) == i))
        fn = sum((np.array(true_classes) == i) & (np.array(pred_classes) != i))
        recall[i] = tp / (tp + fn) if (tp + fn) > 0 else 0
    return recall


In [5]:
def save_confusion_matrix(true_classes, pred_classes, class_names, output_path):
    cm = confusion_matrix(true_classes, pred_classes)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

In [22]:
# Step 5: Main Execution Block
model_path = "kaggle/checkpoint/kaggle/working/Segment-and-Classify/checkpoint_new/best_model.pt"  # Update with your model path
test_dir = r"C:\Users\Xpeedent\Desktop\FPT\SP25\DBM\data_DBM\BrainTumor_Split_mask\test"  # Update with your test directory
output_dir = "pred_output/ori"  # Update with your output directory
confusion_matrix_path = os.path.join(output_dir, 'confusion_matrix.png')

# Load the model
model = load_model(model_path, device)

# Predict masks
true_classes, pred_classes = predict_masks(model, test_dir, output_dir)

# Calculate recall
recall_scores = calculate_recall(true_classes, pred_classes)
print("Recall Scores:", recall_scores)

# Save confusion matrix
save_confusion_matrix(true_classes, pred_classes, ['No_Tumor', 'Glioma', 'Meningioma', 'Pituitary'], confusion_matrix_path)

print("Prediction and evaluation complete!")

C:\Users\Xpeedent\AppData\Local\Temp\ipykernel_20696\3003062792.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


Recall Scores: {0: 0.9875, 1: 0.9692307692307692, 2: 0.93, 3: 0.99}
Prediction and evaluation complete!


# Add _pred

In [23]:
import os
import shutil

def add_pred_to_png_files(root_folder):
    # Counter for renamed files
    renamed_count = 0
    
    # Walk through all folders and subfolders
    for folder_path, _, files in os.walk(root_folder):
        for file in files:
            # Check if file is a PNG
            if file.lower().endswith('.png'):
                # Construct the full file path
                file_path = os.path.join(folder_path, file)
                
                # Create the new filename with "_pred" before ".png"
                base_name = file[:-4]  # Remove the ".png" extension
                new_name = f"{base_name}_pred.png"
                new_file_path = os.path.join(folder_path, new_name)
                
                # Rename the file
                try:
                    shutil.move(file_path, new_file_path)
                    print(f"Renamed: {file} → {new_name}")
                    renamed_count += 1
                except Exception as e:
                    print(f"Error renaming {file}: {e}")
    
    # Print summary
    print(f"\nSummary:")
    print(f"Renamed {renamed_count} PNG files to include '_pred' suffix")
    
    return renamed_count

# Replace with your actual folder path
root_folder = r"C:\Users\Xpeedent\Desktop\Brain\pred_output\ori"

# Run the renaming process
renamed_files = add_pred_to_png_files(root_folder)


Renamed: confusion_matrix.png → confusion_matrix_pred.png
Renamed: Tr-gl_0016.png → Tr-gl_0016_pred.png
Renamed: Tr-gl_0055.png → Tr-gl_0055_pred.png
Renamed: Tr-gl_0057.png → Tr-gl_0057_pred.png
Renamed: Tr-gl_0061.png → Tr-gl_0061_pred.png
Renamed: Tr-gl_0064.png → Tr-gl_0064_pred.png
Renamed: Tr-gl_0084.png → Tr-gl_0084_pred.png
Renamed: Tr-gl_0086.png → Tr-gl_0086_pred.png
Renamed: Tr-gl_0159.png → Tr-gl_0159_pred.png
Renamed: Tr-gl_0175.png → Tr-gl_0175_pred.png
Renamed: Tr-gl_0176.png → Tr-gl_0176_pred.png
Renamed: Tr-gl_0208.png → Tr-gl_0208_pred.png
Renamed: Tr-gl_0216.png → Tr-gl_0216_pred.png
Renamed: Tr-gl_0222.png → Tr-gl_0222_pred.png
Renamed: Tr-gl_0238.png → Tr-gl_0238_pred.png
Renamed: Tr-gl_0242.png → Tr-gl_0242_pred.png
Renamed: Tr-gl_0244.png → Tr-gl_0244_pred.png
Renamed: Tr-gl_0301.png → Tr-gl_0301_pred.png
Renamed: Tr-gl_0359.png → Tr-gl_0359_pred.png
Renamed: Tr-gl_0381.png → Tr-gl_0381_pred.png
Renamed: Tr-gl_0415.png → Tr-gl_0415_pred.png
Renamed: Tr-gl_0419.pn

# Compare folder


In [17]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

def dice_coef_metric(pred_mask, gt_mask, smooth=1e-6):
    """
    Calculate Dice coefficient between tumor regions.
    
    Args:
        pred_mask: Predicted binary mask
        gt_mask: Ground truth binary mask
        smooth: Smoothing factor to avoid division by zero
            
    Returns:
        Dice coefficient score (float)
    """
    # Convert masks to binary
    pred_binary = pred_mask > 0
    gt_binary = gt_mask > 0
    
    # Calculate intersection and union
    intersection = np.sum(pred_binary & gt_binary)
    pred_area = np.sum(pred_binary)
    gt_area = np.sum(gt_binary)
    
    # Calculate Dice coefficient
    denominator = pred_area + gt_area
    
    # Handle edge cases
    if denominator == 0:
        return 1.0  # Both masks are empty
    return (2. * intersection + smooth) / (denominator + smooth)

def calculate_average_dice(test_mask_dir, pred_mask_dir):
    """
    Calculate average Dice scores by comparing test masks with predicted masks.
    
    Args:
        test_mask_dir: Directory containing ground truth masks
        pred_mask_dir: Directory containing predicted masks
        
    Returns:
        Dictionary with average Dice scores for each class
    """
    # Define class names
    class_names = ['No_Tumor', 'Glioma', 'Meningioma', 'Pituitary']
    
    # Initialize results dictionary
    average_dice_scores = {class_name: [] for class_name in class_names}
    
    # Process each class
    for class_name in class_names:
        gt_mask_dir = os.path.join(test_mask_dir, 'mask', class_name)
        if not os.path.exists(gt_mask_dir):
            print(f"Ground truth mask directory for {class_name} not found, skipping.")
            continue
        
        gt_masks = [f for f in os.listdir(gt_mask_dir) if f.endswith('.png')]
        if not gt_masks:
            print(f"No ground truth masks found for {class_name}, skipping.")
            continue
        
        print(f"Processing {len(gt_masks)} masks for class {class_name}...")
        
        # Process each mask
        for mask_file in tqdm(gt_masks, desc=f"Class {class_name}"):
            gt_path = os.path.join(gt_mask_dir, mask_file)
            gt_mask = np.array(Image.open(gt_path).convert('L').resize((256, 256)))
            
            # Normalize ground truth mask if needed
            if gt_mask.max() > 3:
                gt_mask = (gt_mask / 255 * 3).astype(np.uint8)
            
            # Find corresponding predicted mask
            pred_path = os.path.join(pred_mask_dir, class_name, f"{os.path.splitext(mask_file)[0]}_pred.png")
            if not os.path.exists(pred_path):
                print(f"No prediction found for {mask_file}, skipping.")
                continue
            
            pred_mask = np.array(Image.open(pred_path).convert('L').resize((256, 256)))
            if pred_mask.max() > 3:
                pred_mask = (pred_mask / 85).astype(np.uint8)
            
            # Calculate Dice score
            dice_score = dice_coef_metric(pred_mask, gt_mask)
            average_dice_scores[class_name].append(dice_score)
    
    # Calculate average for each class
    average_results = {class_name: np.mean(scores) if scores else 0 for class_name, scores in average_dice_scores.items()}
    
    return average_results


In [24]:

# Main execution block
def main():
    # Set paths
    test_dir = r"C:\Users\Xpeedent\Desktop\FPT\SP25\DBM\data_DBM\BrainTumor_Split_mask\test" # Update with your test directory
    pred_dir = r"C:\Users\Xpeedent\Desktop\Brain\pred_output\ori"  # Update with your predictions directory
    
    # Calculate average Dice scores
    print("Calculating average Dice scores...")
    average_dice_scores = calculate_average_dice(test_dir, pred_dir)
    
    # Print average Dice scores
    print("\nAverage Dice Scores:")
    for class_name, score in average_dice_scores.items():
        print(f"{class_name}: {score:.4f}")

# Execute the main function
main()


Calculating average Dice scores...
Processing 160 masks for class No_Tumor...


Class No_Tumor: 100%|██████████| 160/160 [00:00<00:00, 279.07it/s]


Processing 65 masks for class Glioma...


Class Glioma: 100%|██████████| 65/65 [00:00<00:00, 110.21it/s]


Processing 100 masks for class Meningioma...


Class Meningioma: 100%|██████████| 100/100 [00:00<00:00, 109.83it/s]


Processing 100 masks for class Pituitary...


Class Pituitary: 100%|██████████| 100/100 [00:00<00:00, 106.86it/s]


Average Dice Scores:
No_Tumor: 1.0000
Glioma: 0.7550
Meningioma: 0.8483
Pituitary: 0.8993
